In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import networkx as nx
from heapq import heappush, heappop
from matplotlib import pyplot as plt
import copy
import time


In [ ]:
from walktrap import *

In [ ]:
######################################################################
# Test partition conversions and Rand-index

edge_list = [[1,5],[1,7],[2,3],[2,4],[2,5],[2,9],[3,4],[3,5],[3,6],[3,8],[4,7],[5,6],[6,7],[6,10],[7,8],[8,10],[9,10],[9,11],[9,12],[9,13],[9,15],[10,11],[10,15],[12,14],[13,14],[13,15],[14,15],[15,16]]
G = edge_list_to_graph(edge_list)
G = nx.convert_node_labels_to_integers(G)
pos = nx.spring_layout(G)
nx.draw(G, pos)

In [ ]:


t = 2
parts, coms, _, Qs = walktrap(G, t) 
Qmax_index = np.argmax(Qs)
my_best_part = partition_to_plot(coms, parts[Qmax_index])
#nx.draw(G, pos, node_color= my_best_part.values())
nx.draw(G, pos, node_color= list(my_best_part.values()))




In [ ]:
print(my_best_part)
print(partition_dict_to_sets(my_best_part))
print(partition_set_to_sets(coms, parts[Qmax_index]))

In [ ]:


louvain_best_part = compare_with_Louvain(G)
nx.draw(G, pos, node_color= list(louvain_best_part.values()))
plt.show()
print(louvain_best_part)
print(partition_dict_to_sets(louvain_best_part))

print(calculate_rand_index(partition_dict_to_sets(my_best_part), partition_dict_to_sets(louvain_best_part)))
print(calculate_rand_index(partition_dict_to_sets(my_best_part), partition_dict_to_sets(my_best_part)))

In [ ]:
######################################################################
# Experiments: real graphs
######################################################################

######################################################################
# 1.) Example 16-node graph from the paper
######################################################################
edge_list = [[1,5],[1,7],[2,3],[2,4],[2,5],[2,9],[3,4],[3,5],[3,6],[3,8],[4,7],[5,6],[6,7],[6,10],[7,8],[8,10],[9,10],[9,11],[9,12],[9,13],[9,15],[10,11],[10,15],[12,14],[13,14],[13,15],[14,15],[15,16]]
G = edge_list_to_graph(edge_list)
compare_algos(G, 2)

In [ ]:
######################################################################
# 2.) Experiments on real-graph: Zachary’s Karate Club graph
######################################################################
G = nx.karate_club_graph()
compare_algos(G, 4)

In [ ]:
######################################################################
# 3.) Experiments on real-graph: The college football network
######################################################################
G = nx.read_gml('./../Datasets/football/football.gml')
compare_algos(G, 12)

In [ ]:
######################################################################
# 4.) Experiments on real-graph: Les Miserables network
######################################################################
G = nx.read_gml('./../Datasets/lesmis/lesmis.gml')
compare_algos(G, 6)

In [ ]:
######################################################################
# 5.) Experiments on real-graph: Dolphins network
######################################################################
G = nx.read_gml('./../Datasets/dolphins/dolphins.gml')
compare_algos(G, 2)

In [ ]:
########################################################################################
# Experiments on random graphs: random partition graph: homogenous/heterogenous
########################################################################################
# A partition graph is a graph of communities with sizes defined by s in sizes. 
# Nodes in the same group are connected with probability p_in 
# and nodes of different groups are connected with probability p_out.

n_range = [1000.]
# n_range = [30., 100., 200., 300., 500.]
n_range = [30., 100., 200., 300., 500., 700., 1000.]
ts = [2, 5, 8]
heterogeneous = True

p_in_range = [0.5, 0.4]
p_out_range = [0.1, 0.2]
gamma_range = [0.5, 0.4, 0.3]
num_runs = len(p_in_range) * len(p_out_range) * len(gamma_range)

algos = ['WT2', 'WT5', 'WT8', 'LO']
R = {'WT2': [], 'WT5': [], 'WT8': [], 'LO': [], 'MC': []}
tr = {'WT2': [], 'WT5': [], 'WT8': [], 'LO': [], 'MC': []}
acc = {'WT2': [], 'WT5': [], 'WT8': [], 'LO': [], 'MC': []}

for n in n_range:

    K_range = [int(np.ceil(n ** gamma)) for gamma in gamma_range] # number of communities K = n^gamma

    R_current = {'WT2': 0., 'WT5': 0., 'WT8': 0., 'LO': 0., 'MC': 0.}
    tr_current = {'WT2': 0., 'WT5': 0., 'WT8': 0., 'LO': 0., 'MC': 0.}
    acc_current = {'WT2': 0., 'WT5': 0., 'WT8': 0., 'LO': 0., 'MC': 0.}

    for K_true in K_range:
        
        if heterogeneous:
            sizes = np.random.uniform(0.,1.,K_true)
            sizes = (n * sizes / np.sum(sizes)).astype(int)
        else:
            sizes = np.repeat([int(np.ceil(n / K_true))], K_true)
            
        for p_in in p_in_range:
            for p_out in p_out_range:
                
                # Generate graph
                while True:
                    G = nx.random_partition_graph(sizes, p_in, p_out)
                    if nx.is_connected(G): # Ensure the graph is connected
                        break
                true_part = G.graph['partition']

                # Run all algos
                for t in ts:
                    start_time = time.time()
                    parts, coms, _, Qs = walktrap(G, t) 
                    tr_current['WT'+str(t)] += time.time() - start_time
                    Qmax_index = np.argmax(Qs)
                    K_pred = len(Qs) - Qmax_index
                    if K_pred == K_true:
                        acc_current['WT'+str(t)] += 1.
                    R_current['WT'+str(t)] += calculate_rand_index(true_part, partition_set_to_sets(coms, parts[Qmax_index]))

                louvain_best_part = partition_dict_to_sets(compare_with_Louvain(G, verbose=False))
                if len(louvain_best_part) == K_true:
                    acc_current['LO'] += 1.
                R_current['LO'] += calculate_rand_index(true_part, louvain_best_part)
                               
    # Average Rand-index R' over all num_runs runs
    for algo in algos:
        R[algo].append(R_current[algo] / num_runs)
        tr[algo].append(tr_current[algo] / num_runs)
        acc[algo].append(acc_current[algo] / num_runs)

In [ ]:
# Plot the results R'(n) and acc(n) for the 5 algorithms

n_range = [30., 100., 200., 300., 500., 700., 1000.]
algos = ['WT2', 'WT5', 'WT8', 'LO', 'MC']
mc_data = np.load('MC_random.npz')
# R['MC'] = mc_data['R'][:-2]
# tr['MC'] = mc_data['tr'][:-2]
# acc['MC'] = mc_data['acc'][:-2]
R['MC'] = mc_data['R']
tr['MC'] = mc_data['tr']
acc['MC'] = mc_data['acc']

plt.figure()
for algo in algos:
    plt.plot(n_range, R[algo], 'o-', label=algo)
plt.xlabel('Number of nodes (n)')
plt.ylabel("Corrected Rand-index (R')")
plt.title('Corrected Rand-index vs graph size')
plt.xscale('log')
plt.legend()
plt.show()

plt.figure()
for i, algo in enumerate(algos):
    plt.bar(np.array(n_range) - 24. + i*12., acc[algo], width=12., align='center', label=algo)
#     plt.xscale('log')
plt.xlabel('Number of nodes (n)')
plt.ylabel("Accuracy (correctly detected number of communities)")
plt.title('Correctly identified number of communities vs graph size')
plt.legend(loc='best')
plt.show()

# plt.figure()
# for algo in algos:
#     plt.plot(n_range, R[algo], 'o-', label=algo)
# plt.xlabel('Number of nodes (n)')
# plt.ylabel("Corrected Rand-index (R')")
# plt.title('Corrected Rand-index vs graph size')
# # plt.xscale('log')
# plt.legend()
# plt.show()

# plt.figure()
# for algo in algos:
#     plt.plot(n_range, tr[algo], 'o-', label=algo)
# plt.xlabel('Number of nodes (n)')
# plt.ylabel("Running time")
# plt.title('Running time vs graph size')
# plt.xscale('log')
# plt.yscale('log')
# plt.legend()
# plt.show()



In [ ]:
import numpy as np
import networkx as nx
import markov_clustering as mc
import random
import time

######################################################################
# Comparison with Markov Clustering algorithm: return best partition
######################################################################

def compare_with_MC(G, K_true):
	G = nx.convert_node_labels_to_integers(G)
	print ("Ground truth: ",K_true, " communities")
	start_time = time.time()
	matrix = nx.to_scipy_sparse_matrix(G)
	result = mc.run_mcl(matrix)           # run MCL with default parameters
	clusters = mc.get_clusters(result)    # get clusters
	mc_time = time.time() - start_time
	
	Q = 0. # modularity
	G_total_weight = G.number_of_edges()
	part = {}
	for i, c in enumerate(clusters):
		internal_weight = 0.
		total_weight = 0.
		for v in c:
			part[v] = i
			for w, _ in G.adj[v].items():
				if w in c:
					internal_weight += 1.	# internal edge
				else:
					total_weight += 0.5		# external edge

		internal_weight = internal_weight / 2 # because each edge was counted twice
		total_weight += internal_weight
		Q += ( internal_weight - (total_weight*total_weight/G_total_weight) ) / G_total_weight

	print ("\tOptimal number of communities: K = ", len(np.unique(list(part.values()))))
	print ("\tRuntime: ", mc_time, " seconds")
	print ("\tModularity: ", Q)
	return part # partition as dictionary, suitable for plotting

######################################################################
# Calculate Rand index for partitions P1 and P2, given lists of sets
######################################################################
def calculate_rand_index(P1, P2):
    N = 0
    sum_intersect = 0.
    sum_C1 = 0.
    sum_C2 = np.sum([len(s)**2 for s in P2])
    for s1 in P1:
        N += len(s1)
        sum_C1 += len(s1)**2
        for s2 in P2:
            sum_intersect += len(s1.intersection(s2))**2
    return (N*N*sum_intersect - sum_C1*sum_C2) / ( 0.5*N*N*(sum_C1 + sum_C2) - sum_C1*sum_C2)

######################################################################
# Convert edge list to Networkx Graph
######################################################################
def edge_list_to_graph(edges, verbose=False):
    G = nx.Graph()
    G.add_edges_from(edges)
    if verbose:
        print (G.number_of_edges(), " edges, ", G.number_of_nodes(), " nodes")
    return G

print ("Markov Clustering algorithm:")

def eval_MC_on_real():
	edge_list = [[1,5],[1,7],[2,3],[2,4],[2,5],[2,9],[3,4],[3,5],[3,6],[3,8],[4,7],[5,6],[6,7],[6,10],[7,8],[8,10],[9,10],[9,11],[9,12],[9,13],[9,15],[10,11],[10,15],[12,14],[13,14],[13,15],[14,15],[15,16]]
	G = edge_list_to_graph(edge_list)
	compare_with_MC(G, 2)
	print()
	G = nx.karate_club_graph()
	compare_with_MC(G, 4)
	print()

	G = nx.read_gml('./../Datasets/football/football.gml')
	compare_with_MC(G, 12)
	print()

	G = nx.read_gml('./../Datasets/lesmis/lesmis.gml')
	compare_with_MC(G, 6)
	print()

	G = nx.read_gml('./../Datasets/dolphins/dolphins.gml')
	compare_with_MC(G, 2)
	print()

def eval_MC_on_random(heterogeneous=False):

	n_range = [30., 100., 200., 300., 500., 700., 1000.]

	p_in_range = [0.5, 0.4]
	p_out_range = [0.1, 0.2]
	gamma_range = [0.5, 0.4, 0.3]	
	# p_in_range = [0.5, 0.4, 0.3]
	# p_out_range = [0.1]
	# gamma_range = [0.5, 0.4, 0.3]
	num_runs = len(p_in_range) * len(p_out_range) * len(gamma_range)

	R = []
	tr = []
	acc = []

	for n in n_range:

		K_range = [int(np.ceil(n ** gamma)) for gamma in gamma_range] # number of communities K = n^gamma


		R_current = 0.
		tr_current = 0.
		acc_current = 0.

		for K_true in K_range:

			if heterogeneous:
				sizes = np.random.uniform(0.,1.,K_true)
				sizes = (n * sizes / np.sum(sizes)).astype(int)
			else:
				sizes = np.repeat([int(np.ceil(n / K_true))], K_true)

			for p_in in p_in_range:
				for p_out in p_out_range:

					# Generate graph
					while True:
						G = nx.random_partition_graph(sizes, p_in, p_out)
						if nx.is_connected(G): # Ensure the graph is connected
							break
					true_part = G.graph['partition']


					start_time = time.time()
					result = mc.run_mcl(nx.to_scipy_sparse_matrix(G))       # run MCL with default parameters
					clusters = mc.get_clusters(result)    # get clusters
					tr_current += time.time() - start_time
					K_pred = len(clusters)
					if K_pred == K_true:
						acc_current += 1.


					list_of_sets = []
					for c in clusters:
						list_of_sets.append(set(c))

					R_current += calculate_rand_index(true_part, list_of_sets)

		# Average Rand-index R' over all num_runs runs
		R.append(R_current / num_runs)
		tr.append(tr_current / num_runs)
		acc.append(acc_current / num_runs)

	np.savez('MC_random_hetero.npz', R=R, tr=tr, acc=acc)

eval_MC_on_random(True)

#eval_MC_on_real()